# Project 2

    ## Objective
    
            Investigate the trend of food poisoning incidents caused by New York City restaurants from 2010 to current.
     
    ## Source Data
    
      NYC Heath inspection data-https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j
       
      NYC food poisoning complaints - https://data.cityofnewyork.us/resource/gjkf-etq5.json

        
    ## Process    
    
        1. Extract data from websites
        2. Transform data
        3. Load data   
            

# Extract Data

    Install and import neccessary libraries and functions 

In [ ]:
!pip install sodapy

In [1]:
import pandas as pd
from sodapy import Socrata

In [2]:
client = Socrata("data.cityofnewyork.us", None)

    
 ### Extract Jason files from predefined datasets  
  

In [3]:
            # Extract First 400000 Health inspection results in JSON formant from API to Python list of dict by sodapy

inspections = client.get("9w7m-hzhe", limit=400000)            
    
            # Extract First 30000 records of reported food poisoning incidents
            # results in JSON formant from API to Python list of dict by sodapy
        
food_df = client.get("gjkf-etq5", limit=30000)                 



# Tranform Data


    To make this data useful we need to connect the health inspection data to the food poisoning incident data thru a 
    common "Key". We choose to use a combination of the Borough name and Street address (In New York City the five boroughs making up the city have duplicate street names and numbers - adding the borough to the street address makes the "key" unique.
        

### Inspection dataset

In [4]:
            # Convert to pandas DataFrame

inspections_df = pd.DataFrame.from_records(inspections)  


In [5]:
            # Rename columns       

inspections = inspections_df.rename(columns={"action": "Action","boro": "Borough","building": "Street_Number",
                                          "camis": "Restaurant_ID","critical_flag": "Critical_Flag",
                                          "cuisine_description": "Cuisine","dba": "Name","grade": "Grade",
                                          "inspection_date": "Inspection_Date","inspection_type": "Inspection_Type",
                                          "score": "Score","street": "Street","violation_code": "Violation_Code",
                                          "violation_description": "Violation_Description","zipcode": "Zip"})

In [6]:
            # Make copy of Street andress before manipulating

inspections['Orig_Street'] = inspections['Street']

In [7]:
            # Convert inspection Date to datetime fomat
    
inspections['Inspection_Date'] = pd.to_datetime(inspections["Inspection_Date"])


            The inspection data set used a "-" inconsistently in the "Street Number" of the address.
            We decided to remove it all together from both data sets to be consistant
                

In [8]:
            # Remove "-" from the Street Number
 
                # split the "street number into columns at the "-"    
                
x = inspections['Street_Number'].str.split('-', expand=True).rename(columns = lambda x: "string"+str(x+1))

                # define 3 new created columns as strings in order to replace "None" with a " "

x['string1'] = x['string1'].astype(str)
x['string2'] = x['string2'].astype(str)
x['string3'] = x['string3'].astype(str)

j = x.replace(['None'],[' '])

                # concatenate columns back to one 'street number without th "-"

j['Street_Number'] = j['string1'] + j['string2'] +j['string3']

                # strip any trailing blank spaces from column        

j = j.applymap(lambda x: x.strip() if type(x)==str else x)



In [9]:
          # Remove double "white spaces from 'Street Name'  


inspections['Street'] = inspections.Street.astype(str)         # Define Street as a string in order to split/join

street = inspections['Street']                                 # Define variables for loop    
x = []                                                         

for i in street:                                               # Take out double white spaces
        x.append(' '.join(i.split()))                            
    
y = pd.Series(x)                                               # Add new 'Adjusted Street' column to df
inspections.insert(loc=0, column='Adj_Street', value=y)         


In [10]:
        # The Street names from both data sets were inconsistent:
        # with regard to :
            
                # the spelling of thorough-fare names,
                # representation of number named streets
                # directional notation
                
        # we first split the Address into columns with "split - expand" 
        # then replaced all "found unique versions" with consistent names

In [11]:
                # Split 'Street address into colums'
                
x = inspections['Adj_Street'].str.split(' ', expand=True).rename(columns = lambda x: "string"+str(x+1))


In [12]:
                # Replace thorough-fare names

y = x.replace(["AIR","AIRPOR","AIRPORT","AIRPOT","airport","ARPT","Airport","AMERICAS","AV","AVE","AVE.","AVENUE","AVEUE",
         "Ave","Ave","Avenue","ave","avenue","BLVD","BLVD.","BOULEVA","BOULEVARD","Blvd","BROADFWAY","BROADWAY",
         "Broadwaty","Broadway","BL","BLDG","BUIL","BUILDING","CENTER","CTR","Center","CIR","CIRCLE","CONCOURS",
         "CONCOURSE","CONCRS","COURT","CT","DR","DRIVE","EXPRESSWAY","EXPWY","EXPY","EXT","EXTENSION","HIGHWAY",
         "HWY","IS","ISLAND","LANE","LN","PARK","PK","PARKWA","PARKWAY","PKWY","PLC","PL","PLACE","PLAZA","PLZ",
         "Plaza","RD","ROAD","SQ","SQUARE","SREET","ST","ST.","STEET","STREET","STREET.","STRRET","STTREET","St",
         "Street","st","street]","TNPK","TPKE","TURNPIKE","TER","TERRACE","TERM","TERMINAL"],
              ["AIRPORT","AIRPORT",
         "AIRPORT","AIRPORT","AIRPORT","AIRPORT","AIRPORT","AMERICAS","AVENUE","AVENUE","AVENUE",
         "AVENUE","AVENUE","AVENUE","AVENUE","AVENUE","AVENUE","AVENUE","BOULEVARD","BOULEVARD","BOULEVARD",
         "BOULEVARD","BOULEVARD","BROADWAY","BROADWAY","BROADWAY","BROADWAY","BUILDING","BUILDING","BUILDING",
         "BUILDING","CENTER","CENTER","CENTER","CIRCLE","CIRCLE","CONCOURSE","CONCOURSE","CONCOURSE","COURT",
         "COURT","DRIVE","DRIVE","EXPRESSWAY","EXPRESSWAY","EXPRESSWAY","EXTENSION","EXTENSION","HIGHWAY",
         "HIGHWAY","ISLAND","ISLAND","LANE","LANE","PARK","PARK","PARKWAY","PARKWAY","PARKWAY","PLACE","PLACE",
         "PLACE","PLAZA","PLAZA","PLAZA","ROAD","ROAD","SQUARE","SQUARE","STREET","STREET","STREET","STREET",
         "STREET","STREET","STREET","STREET","STREET","STREET","STREET","STREET","TURNPIKE","TURNPIKE","TURNPIKE",
         "TERRACE","TERRACE","TERMINAL","TERMINAL"])

In [13]:
                # Replace numbered named streets

z = y.replace(["1ST","2ND","3RD","4TH","5TH","6TH","7TH","8TH","9TH","10TH","11TH","12TH","13TH","14TH","15TH","16TH",
               "17TH","18TH","19TH","20TH","21ST","22ND","23RD","24TH","25TH","26TH","27TH","28TH","29TH","30TH","31ST",
               "32ND","33RD","34TH","35TH","36TH","37TH","38TH","39TH","40TH","41ST","42ND","43RD","44TH","45TH","46TH",
               "47TH","48TH","49TH","50TH","51ST","52ND","53RD","54TH","55TH","56TH","57TH","58TH","59TH","60TH","61ST",
               "62ND","63RD","64TH","65TH","66TH","67TH","68TH","69TH","70TH","71ST","72ND","73RD","74TH","75TH","76TH",
               "77TH","78TH","79TH","80TH","81ST","82ND","83RD","84TH","85TH","86TH","87TH","88TH","89TH","90TH","91ST",
               "92ND","93RD","94TH","95TH","96TH","97TH","98TH","99TH","100TH","101ST","102ND","103RD","104TH","105TH",
               "106TH","107TH","108TH","109TH","110TH","111TH","112TH","113TH","114TH","115TH","116TH","117TH","118TH",
               "119TH","120TH","121ST","122ND","123RD","124TH","125TH","126TH","127TH","128TH","129TH","130TH","131ST",
               "132ND","133RD","134TH","135TH","136TH","137TH","138TH","139TH","140TH","141ST","142ND","143RD","144TH",
               "145TH","146TH","147TH","148TH","149TH","150TH","151ST","152ND","153RD","154TH","155TH","156TH","157TH",
               "158TH","159TH","160TH","161ST","162ND","163RD","164TH","165TH","166TH","167TH","168TH","169TH","170TH",
               "171ST","172ND","173RD","174TH","175TH","176TH","177TH","178TH","179TH","180TH","181ST","182ND","183RD",
               "184TH","185TH","186TH","187TH","188TH","189TH","190TH","191ST","192ND","193RD","194TH","195TH","196TH",
               "197TH","198TH","199TH","200TH","201ST","202ND","203RD","204TH","205TH","206TH","207TH","208TH","209TH",
               "210TH","211TH","212TH","213TH","214TH","215TH","216TH","217TH","218TH","219TH","220TH","221ST","222ND",
               "223RD","224TH","225TH","226TH","227TH","228TH","229TH","230TH","231ST","232ND","233RD","234TH","235TH",
               "236TH","237TH","238TH","239TH","240TH","241ST","242ND","243RD","244TH","245TH","246TH","247TH","248TH",
               "249TH","250TH","251ST","252ND","253RD","254TH","255TH","256TH","257TH","258TH","259TH","260TH","261ST",
               "262ND","263RD","264TH","265TH","266TH","267TH","268TH","269TH","270TH","271ST","272ND","273RD","274TH",
               "275TH","276TH","277TH","278TH","279TH","280TH","281ST","282ND","283RD","284TH","285TH","286TH","287TH",
               "288TH","289TH","290TH","291ST","292ND","293RD","294TH","295TH","296TH","297TH","298TH","299TH","300TH"],
               [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,
                39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,
                74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,
                107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,
                133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,
                159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,
                185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,
                211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,
                237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,
                263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,
                289,290,291,292,293,294,295,296,297,298,299,300])


In [14]:
                # Replaced directional notation

a = z.replace(["N","N.","Nr","Nr.","Nth","Nth.","Nrth","Nrth.","S","S.","Sth","Sth.","E","E.","Est",
               "Est.","W","W.","Wst","Wst."],["NORTH","NORTH","NORTH","NORTH","NORTH","NORTH","NORTH",
               "NORTH","SOUTH","SOUTH","SOUTH","SOUTH","EAST","EAST","EAST","EAST","WEST","WEST","WEST","WEST"])


In [15]:
                # Converted columns to strings inorder to replace "None" with " "

a['string1']= a['string1'].astype(str)
a['string2']= a['string2'].astype(str)
a['string3']= a['string3'].astype(str)
a['string4']= a['string4'].astype(str)
a['string5']= a['string5'].astype(str)
a['string6']= a['string6'].astype(str)

p = a.replace(['None'],[' '])

In [16]:
                # Concatenated split columns back to full street address

p['Adj_New'] = j['Street_Number']+' '+p['string1']+' '+p['string2']+' '+p['string3']+' '+p['string4']+' '+p['string5']+' '+p['string6']

In [17]:
                # Add 'Final Street' column to inspection df

inspections['Final_Street'] = p['Adj_New']

In [18]:
                # concatenate 'Final Street' with 'Borough' to create unique key to join tables
 
inspections['Address_ID'] = inspections['Borough'] + " | " + inspections['Final_Street']

In [19]:
                # Reoder the columns

inspections = inspections[["Restaurant_ID","Address_ID","Inspection_Date","Name",
                           "Cuisine","Final_Street","Borough","Zip","Score",
                           "Violation_Code","Violation_Description"]]

In [20]:
                # Trim white space 

inspections = inspections.applymap(lambda x: x.strip() if type(x)==str else x)

In [21]:
                # Drop 'N/A' rows       

inspections = inspections[inspections.Zip != 'N/A']
inspections = inspections[inspections.Name != 'N/A']


In [22]:
import numpy as np

In [23]:
                # Replace blanks with 'nan'

inspections['Score'].replace('', np.nan, inplace=True)
inspections['Violation_Description'].replace('', np.nan, inplace=True)  

                 # Drop 'N/A' rows 
    
inspections.dropna(subset=['Violation_Description'], inplace=True)        
inspections.dropna(subset=['Score'], inplace=True)

In [24]:
                # Set 'Restaurant_ID' as index

inspections.set_index('Restaurant_ID', inplace=True)

In [25]:
                # Save Transforme DataFrame to csv file

inspections.to_csv('c:/LearnPython/final_inspections.csv',index=False, header=True)

### Food dataset

    (perform same tranformation as above to food dataset)

In [26]:
                # Convert to pandas DataFrame

food_df = pd.DataFrame.from_records(food_df)   


In [27]:
                # Rename columns       
        
food = food_df.rename(columns={"created_date": "Incident_Date","complaint_type": "Complaint","address_type": "Address_Type",
                               "incident_address": "Street_Address","street_name": "Street","borough": "Borough",
                               "city": "City","incident_zip": "Zip","descriptor": "Descriptor","unique_key":"id",
                                "longitude": "Longitude", "latitude":"Latitude"})

In [28]:

                # Transform 'Incident Date' to date format and remove time portion
    

food['Incident_Date'] = pd.to_datetime(food["Incident_Date"])

food['Incident_Date']= food['Incident_Date'].astype(str)
x = food['Incident_Date'].str.split(' ', expand=True).rename(columns = lambda x: "string"+str(x+1))
x['string1'] = pd.to_datetime(x['string1'])

food['Incident_Date'] = x['string1']

In [29]:
                # Save 'Origina Street' name for comparison                    

food['Orig_Street'] = food['Street_Address']

In [30]:
                # Remove white space  
    
food['Street_Address'] = food.Street_Address.astype(str)       # Define Street as a string in order to split/join

y = food['Street_Address']                                     # Define variables for loop    
x = []                                                         

for i in y:                                                    # Take out double white spaces
        x.append(' '.join(i.split()))                             
    
y = pd.Series(x)                                               # Add new 'Adjusted Street' column to df
food.insert(loc=0, column='Adj_Street', value=y)                 
   

In [31]:
                # Expand columns inorder to replace inconsistent names with consistent

x = food['Adj_Street'].str.split(' ', expand=True).rename(columns = lambda x: "string"+str(x+1))

In [32]:
                # Replace thorough-fare names

y = x.replace(["AIR","AIRPOR","AIRPORT","AIRPOT","airport","ARPT","Airport","AMERICAS","AV","AVE","AVE.","AVENUE",
               "AVEUE","Ave","Ave","Avenue","ave","avenue","BLVD","BLVD.","BOULEVA","BOULEVARD","Blvd","BROADFWAY",
               "BROADWAY","Broadwaty","Broadway","BL","BLDG","BUIL","BUILDING","CENTER","CTR","Center","CIR","CIRCLE",
               "CONCOURS","CONCOURSE","CONCRS","COURT","CT","DR","DRIVE","EXPRESSWAY","EXPWY","EXPY","EXT","EXTENSION",
               "HIGHWAY","HWY","IS","ISLAND","LANE","LN","PARK","PK","PARKWA","PARKWAY","PKWY","PLC","PL","PLACE",
               "PLAZA","PLZ","Plaza","RD","ROAD","SQ","SQUARE","SREET","ST","ST.","STEET","STREET","STREET.","STRRET",
               "STTREET","St","Street","st","street]","TNPK","TPKE","TURNPIKE","TER","TERRACE","TERM","TERMINAL"],
               ["AIRPORT","AIRPORT","AIRPORT","AIRPORT","AIRPORT","AIRPORT","AIRPORT","AMERICAS","AVENUE","AVENUE",
                "AVENUE","AVENUE","AVENUE","AVENUE","AVENUE","AVENUE","AVENUE","AVENUE","BOULEVARD","BOULEVARD",
                "BOULEVARD","BOULEVARD","BOULEVARD","BROADWAY","BROADWAY","BROADWAY","BROADWAY","BUILDING","BUILDING",
                "BUILDING","BUILDING","CENTER","CENTER","CENTER","CIRCLE","CIRCLE","CONCOURSE","CONCOURSE","CONCOURSE",
                "COURT","COURT","DRIVE","DRIVE","EXPRESSWAY","EXPRESSWAY","EXPRESSWAY","EXTENSION","EXTENSION",
                "HIGHWAY","HIGHWAY","ISLAND","ISLAND","LANE","LANE","PARK","PARK","PARKWAY","PARKWAY","PARKWAY",
                "PLACE","PLACE","PLACE","PLAZA","PLAZA","PLAZA","ROAD","ROAD","SQUARE","SQUARE","STREET","STREET",
                "STREET","STREET","STREET","STREET","STREET","STREET","STREET","STREET","STREET","STREET","TURNPIKE",
                "TURNPIKE","TURNPIKE","TERRACE","TERRACE","TERMINAL","TERMINAL"])

          

In [33]:
                # Replace numbered named streets

q = y.replace(["1ST","2ND","3RD","4TH","5TH","6TH","7TH","8TH","9TH","10TH","11TH","12TH","13TH","14TH","15TH","16TH",
               "17TH","18TH","19TH","20TH","21ST","22ND","23RD","24TH","25TH","26TH","27TH","28TH","29TH","30TH","31ST",
               "32ND","33RD","34TH","35TH","36TH","37TH","38TH","39TH","40TH","41ST","42ND","43RD","44TH","45TH","46TH",
               "47TH","48TH","49TH","50TH","51ST","52ND","53RD","54TH","55TH","56TH","57TH","58TH","59TH","60TH","61ST",
               "62ND","63RD","64TH","65TH","66TH","67TH","68TH","69TH","70TH","71ST","72ND","73RD","74TH","75TH","76TH",
               "77TH","78TH","79TH","80TH","81ST","82ND","83RD","84TH","85TH","86TH","87TH","88TH","89TH","90TH","91ST",
               "92ND","93RD","94TH","95TH","96TH","97TH","98TH","99TH","100TH","101ST","102ND","103RD","104TH","105TH",
               "106TH","107TH","108TH","109TH","110TH","111TH","112TH","113TH","114TH","115TH","116TH","117TH","118TH",
               "119TH","120TH","121ST","122ND","123RD","124TH","125TH","126TH","127TH","128TH","129TH","130TH","131ST",
               "132ND","133RD","134TH","135TH","136TH","137TH","138TH","139TH","140TH","141ST","142ND","143RD","144TH",
               "145TH","146TH","147TH","148TH","149TH","150TH","151ST","152ND","153RD","154TH","155TH","156TH","157TH",
               "158TH","159TH","160TH","161ST","162ND","163RD","164TH","165TH","166TH","167TH","168TH","169TH","170TH",
               "171ST","172ND","173RD","174TH","175TH","176TH","177TH","178TH","179TH","180TH","181ST","182ND","183RD",
               "184TH","185TH","186TH","187TH","188TH","189TH","190TH","191ST","192ND","193RD","194TH","195TH","196TH",
               "197TH","198TH","199TH","200TH","201ST","202ND","203RD","204TH","205TH","206TH","207TH","208TH","209TH",
               "210TH","211TH","212TH","213TH","214TH","215TH","216TH","217TH","218TH","219TH","220TH","221ST","222ND",
               "223RD","224TH","225TH","226TH","227TH","228TH","229TH","230TH","231ST","232ND","233RD","234TH","235TH",
               "236TH","237TH","238TH","239TH","240TH","241ST","242ND","243RD","244TH","245TH","246TH","247TH","248TH",
               "249TH","250TH","251ST","252ND","253RD","254TH","255TH","256TH","257TH","258TH","259TH","260TH","261ST",
               "262ND","263RD","264TH","265TH","266TH","267TH","268TH","269TH","270TH","271ST","272ND","273RD","274TH",
               "275TH","276TH","277TH","278TH","279TH","280TH","281ST","282ND","283RD","284TH","285TH","286TH","287TH",
               "288TH","289TH","290TH","291ST","292ND","293RD","294TH","295TH","296TH","297TH","298TH","299TH","300TH"],
               [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,
                39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,
                74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,
                107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,
                133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,
                159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,
                185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,
                211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,
                237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,
                263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,
                289,290,291,292,293,294,295,296,297,298,299,300])

In [34]:
              # Replaced directional notation

a = q.replace(["N","N.","Nr","Nr.","Nth","Nth.","Nrth","Nrth.","S","S.","Sth","Sth.","E","E.","Est",
               "Est.","W","W.","Wst","Wst."],["NORTH","NORTH","NORTH","NORTH","NORTH","NORTH","NORTH",
               "NORTH","SOUTH","SOUTH","SOUTH","SOUTH","EAST","EAST","EAST","EAST","WEST","WEST","WEST","WEST"])

In [35]:
                

x = a['string1'].str.split(' ', expand=True).rename(columns = lambda x: "string"+str(x+1))

In [36]:
                # Converted columns to strings inorder to replace "None" with " "

a['string1']= a['string1'].astype(str)
a['string2']= a['string2'].astype(str)
a['string3']= a['string3'].astype(str)
a['string4']= a['string4'].astype(str)
a['string5']= a['string5'].astype(str)


p = a.replace(['None'],[' '])

In [37]:
                # Remove "-" from the Street Number as above in inspection data se
 
x = a['string1'].str.split('-', expand=True).rename(columns = lambda x: "string"+str(x+1))

x['string2'] = x['string2'].astype(str)
j = x.replace(['None'],[' '])

j = j.applymap(lambda x: x.strip() if type(x)==str else x)

j['Street_Number'] = j['string1'] + j['string2']

p['string1'] = j['Street_Number']

In [38]:
                # Concatenated split columns back to full street address

p['Adj_New'] = p['string1'] + ' ' + p['string2'] + ' ' + p['string3'] + ' ' + p['string4'] + ' ' + p['string5']


In [39]:
               # Add 'Final address to food df' 

food['Final_Street'] = p['Adj_New']


In [40]:
               # concatenate 'Final Street' with 'Borough' to create unique key to join tables

food['Address_Id'] = food['Borough'] + " | " + food['Final_Street']


In [41]:
                # Create new 'Weekday' column form date for weekday analysis


from datetime import date
import calendar

y = food['Incident_Date']                                     # Define variables for loop    
x = []                                                        

for i in y:                                                    # Take out double white spaces
        x.append(calendar.day_name[i.weekday()])                            
    
y = pd.Series(x)                                               # Add new 'Adjusted Street' column to df
food.insert(loc=0, column='Weekday', value=y)                

In [42]:
                # Re-order columns

food = food[["id","Address_Id","Incident_Date","Weekday","Complaint",
             "Final_Street","Borough","Zip"]]

In [43]:
                # Set 'id' as index

food.set_index('id', inplace=True)

In [44]:
                # final strip of trailing white space

food = food.applymap(lambda x: x.strip() if type(x)==str else x)

In [45]:
                # delet 'N/A' rows

food = food[food.Zip != 'N/A']

In [46]:
                # Save Transforme DataFrame to csv file

food.to_csv('c:/LearnPython/final_food.csv', header=True)

# Load DataFrames into database

In [ ]:
import pymysql
pymysql.install_as_MySQLdb()

In [ ]:
from sqlalchemy import create_engine

In [ ]:
                # Create Engine and Pass in MySQL Connection

connection_string = "root:toor@localhost/NewYorkCity_db"
engine = create_engine(f'mysql://{connection_string}')

In [ ]:
                # Confirm tables set up in MySql
    
engine.table_names()

In [ ]:
                # Load data into database tables after first deleting data in tables in mysql

food.to_sql(name='food', con=engine, if_exists='append', index=True)

In [ ]:
                # Verify load successful

pd.read_sql_query('select * from food', con=engine).head()

In [ ]:
                # Load data into database tables after first deleting data in tables in mysql

inspections.to_sql(name='inspection', con=engine, if_exists='append', index=True, chunksize=10000)

In [ ]:
                # Verify load successful

pd.read_sql_query('select * from inspection', con=engine).head(5)